In [23]:
import pandas as pd

adult_df = pd.read_csv("adult.csv")

In [24]:
adult_df

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,22,Private,310152,Some-college,10,Never-married,Protective-serv,Not-in-family,White,Male,0,0,40,United-States,<=50K
32557,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32558,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32559,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K


In [25]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

# Create pension proxy target variable
def create_pension_target(df):
    """
    Create a realistic pension eligibility proxy
    """
    # Higher weights for government jobs and older age
    pension_score = (
        (df['age'] >= 65) * 3 +
        (df['age'].between(55, 64)) * 2 +
        (df['workclass'].isin(['Federal-gov', 'State-gov', 'Local-gov'])) * 3 +
        (df['occupation'].isin(['Prof-specialty', 'Exec-managerial', 'Tech-support'])) * 2 +
        (df['income'] == '>50K') * 2 +
        (df['education.num'] >= 16) * 1 +
        (df['capital.gain'] > 10000) * 1
    )
    
    # Convert to categorical: Low (0-3), Medium (4-6), High (7+)
    df['pension_category'] = pd.cut(pension_score, 
                                   bins=[-1, 3, 6, 20], 
                                   labels=['Low', 'Medium', 'High'])
    
    return df, pension_score

df, pension_scores = create_pension_target(adult_df)
print("Pension Category Distribution:")
print(df['pension_category'].value_counts())

Pension Category Distribution:
pension_category
Low       25430
Medium     5886
High       1245
Name: count, dtype: int64


In [26]:
df

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income,pension_category
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K,Low
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K,Medium
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K,Low
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K,Low
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K,Low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,22,Private,310152,Some-college,10,Never-married,Protective-serv,Not-in-family,White,Male,0,0,40,United-States,<=50K,Low
32557,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K,Low
32558,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K,Low
32559,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K,Low


In [37]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer

categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()

categorical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant',fill_value='Unknown')),
    ('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])

numerical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))
])

transformer = ColumnTransformer(transformers=[
    ('num', numerical_pipeline, numerical_cols),
    ('cat', categorical_pipeline, categorical_cols)
])

transformed_array = transformer.fit_transform(df)

# Get feature names
feature_names = transformer.get_feature_names_out()

# Convert back to DataFrame with proper column names
df_transformed = pd.DataFrame(transformed_array, columns=feature_names)

In [38]:
df_transformed

,num__age,num__fnlwgt,num__education.num,num__capital.gain,num__capital.loss,num__hours.per.week,cat__workclass,cat__education,cat__marital.status,cat__occupation,cat__relationship,cat__race,cat__sex,cat__native.country,cat__income,cat__pension_category
0,90.0,77053.0,9.0,0.0,4356.0,40.0,0.0,11.0,6.0,0.0,1.0,4.0,0.0,39.0,0.0,1.0
1,82.0,132870.0,9.0,0.0,4356.0,18.0,4.0,11.0,6.0,4.0,1.0,4.0,0.0,39.0,0.0,2.0
2,66.0,186061.0,10.0,0.0,4356.0,40.0,0.0,15.0,6.0,0.0,4.0,2.0,0.0,39.0,0.0,1.0
3,54.0,140359.0,4.0,0.0,3900.0,40.0,4.0,5.0,0.0,7.0,4.0,4.0,0.0,39.0,0.0,1.0
4,41.0,264663.0,10.0,0.0,3900.0,40.0,4.0,15.0,5.0,10.0,3.0,4.0,0.0,39.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,22.0,310152.0,10.0,0.0,0.0,40.0,4.0,15.0,4.0,11.0,1.0,4.0,1.0,39.0,0.0,1.0
32557,27.0,257302.0,12.0,0.0,0.0,38.0,4.0,7.0,2.0,13.0,5.0,4.0,0.0,39.0,0.0,1.0
32558,40.0,154374.0,9.0,0.0,0.0,40.0,4.0,11.0,2.0,7.0,0.0,4.0,1.0,39.0,1.0,1.0
32559,58.0,151910.0,9.0,0.0,0.0,40.0,4.0,11.0,6.0,1.0,4.0,4.0,0.0,39.0,0.0,1.0


In [33]:
from sklearn.model_selection import train_test_split

X = df_transformed.drop(['cat_pension_category'], axis=1)
y = df_transformed['cat_pension_category']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)



KeyError: "['pension_category'] not found in axis"

In [30]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd
import numpy as np

def evaluate_model(model, X_test, y_test, model_name="Model"):
    """
    Basic model evaluation function
    """
    # Predictions
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test) if hasattr(model, "predict_proba") else None
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
    
    # Print results
    print(f"╔{'═'*50}╗")
    print(f"║ {model_name:^48} ║")
    print(f"╠{'═'*50}╣")
    print(f"║ {'Accuracy:':<20} {accuracy:.4f} {'│':<5} {' ':20} ║")
    print(f"║ {'Precision:':<20} {precision:.4f} {'│':<5} {' ':20} ║")
    print(f"║ {'Recall:':<20} {recall:.4f} {'│':<5} {' ':20} ║")
    print(f"║ {'F1-Score:':<20} {f1:.4f} {'│':<5} {' ':20} ║")
    print(f"╚{'═'*50}╝")
    
    # Classification report
    print(f"\n📊 Classification Report:")
    print("─" * 60)
    print(classification_report(y_test, y_pred, zero_division=0))
    
    # Confusion matrix
    print(f"\n🎯 Confusion Matrix:")
    print("─" * 40)
    cm = confusion_matrix(y_test, y_pred)
    print(cm)
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'predictions': y_pred,
        'probabilities': y_pred_proba
    }

In [32]:
X_train

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
937,28,Private,303954,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,0,1848,42,United-States,>50K
24931,65,Private,93318,Some-college,10,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,45,United-States,<=50K
9643,52,Local-gov,74784,Masters,14,Married-civ-spouse,Prof-specialty,Wife,White,Female,0,0,40,United-States,>50K
32303,20,?,99891,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K
541,36,Private,156667,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,1902,50,United-States,>50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8138,20,Self-emp-inc,182200,HS-grad,9,Never-married,Machine-op-inspct,Own-child,White,Female,0,0,30,United-States,<=50K
28315,52,Local-gov,153064,Some-college,10,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States,>50K
29542,39,Private,290922,Masters,14,Never-married,Prof-specialty,Own-child,White,Female,0,0,40,United-States,<=50K
24057,19,?,184308,Some-college,10,Married-civ-spouse,?,Wife,White,Female,0,0,30,United-States,<=50K


In [31]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

rf_classifier = RandomForestClassifier(random_state=0, n_jobs=-1)

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}

grid_search = GridSearchCV(
    estimator=rf_classifier,
    param_grid=param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train, y_train)




Fitting 5 folds for each of 162 candidates, totalling 810 fits


ValueError: 
All the 810 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
162 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/michalkawa/heackyeh-bieda-team/server/.venv/lib/python3.13/site-packages/sklearn/model_selection/_validation.py", line 859, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/michalkawa/heackyeh-bieda-team/server/.venv/lib/python3.13/site-packages/sklearn/base.py", line 1365, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/Users/michalkawa/heackyeh-bieda-team/server/.venv/lib/python3.13/site-packages/sklearn/ensemble/_forest.py", line 359, in fit
    X, y = validate_data(
           ~~~~~~~~~~~~~^
        self,
        ^